# 1. YOLO모델 적용 후 동영상 저장

In [ ]:
import cv2
import numpy as np
from imageai.Detection import VideoObjectDetection
import os

execution_path = os.getcwd()

detector = VideoObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath( os.path.join(execution_path , "models/yolo.h5"))
detector.loadModel()

video_path = detector.detectObjectsFromVideo(
    input_file_path=os.path.join(execution_path, "videos/v2/v2_raw.mp4"),
    output_file_path=os.path.join(execution_path, "videos/v2/v2_YOLO"),
    frames_per_second=30, log_progress=True)
print(video_path)

# 2. YOLO모델에서 사람의 수 COUNT후 파일로 저장

In [ ]:
from imageai.Detection import VideoObjectDetection
import os

#YOLO 동영상 저장
execution_path = os.getcwd()
resized = False

def forFrame(frame_number, output_array, output_count, returned_frame):


    this_colors = []
    labels = []
    sizes = []

    counter = 0
    
    file = open('V2_person.txt','a')
    
    if 'person' in output_count:
        file.write(str(output_count['person']) + '\n')
    else:
        file.write(str(0) + '\n')
    file.close()
    
    global resized



video_detector = VideoObjectDetection()
video_detector.setModelTypeAsYOLOv3()
video_detector.setModelPath(os.path.join(execution_path, "models/yolo.h5"))
video_detector.loadModel()

video_detector.detectObjectsFromVideo(input_file_path=os.path.join(execution_path, "videos/termproj_V2.mp4"), 
                                      output_file_path=os.path.join(execution_path, "video_frame_analysis_test") ,  
                                      frames_per_second = 30, per_frame_function=forFrame,  
                                      minimum_percentage_probability=50, 
                                      return_detected_frame=True)

# 3. YOLO동영상을 mtcnn으로 얼굴 인식 + 얼굴의 수 count 후 파일 저장

In [ ]:
import cv2
import numpy as np
from mtcnn.mtcnn import MTCNN
from imageai.Detection import VideoObjectDetection
import os

detector = MTCNN()
cap = cv2.VideoCapture('videos/v2/v2_YOLO.avi')
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = 30

writer = cv2.VideoWriter('videos/v2/v2_YOLO_MTCNN.avi', fourcc, fps, (width, height))

while True: 
    ret, frame = cap.read()
    
    if ret == False: 
        break
        
    file = open('V2_face.txt', 'a')
    result = detector.detect_faces(frame)
    if result != []:
        for person in result:
            bounding_box = person['box']
            keypoints = person['keypoints']
    
            cv2.rectangle(frame,
                          (bounding_box[0], bounding_box[1]),
                          (bounding_box[0]+bounding_box[2], bounding_box[1] + bounding_box[3]),
                          (0,155,255),
                          2)
    
            cv2.circle(frame,(keypoints['left_eye']), 2, (0,155,255), 2)
            cv2.circle(frame,(keypoints['right_eye']), 2, (0,155,255), 2)
            cv2.circle(frame,(keypoints['nose']), 2, (0,155,255), 2)
            cv2.circle(frame,(keypoints['mouth_left']), 2, (0,155,255), 2)
            cv2.circle(frame,(keypoints['mouth_right']), 2, (0,155,255), 2)
        file.write(str(len(result)) + '\n')       #result의 element수가 detect하는 수가 된다. 즉, 얼굴의 개수
    else:
        file.write(str(0) + '\n')
    
    file.close()
    writer.write(frame)
    
   
cap.release()
writer.release()
cv2.destroyAllWindows()